# Calculando SVF para um ponto com Pandas

Neste experimento vamos calcular para um ponto definido o SVF utilizando Pandas. Para isso será utilizado uma técnica de calcular para cada ponto o $\theta$ e o $\phi$ a partir do ponto escolhido e então agrupar pelo particionamento da semiesfera definida

In [381]:
import pandas as pd
import geopandas as gpd
import pdal
import numpy as np

In [382]:
%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime


In [383]:
pipeline="""{
  "pipeline": [
    {
        "type": "readers.las",
        "filename": "arquivos/MDS/MDS_3314-231.laz"
    },
    {
        "type": "filters.sample",
        "radius": 1
    }
  ]
}"""

In [384]:
gdf_observ = gpd.read_file('arquivos/pontos_validacao/pontos_preliminares.gpkg')

In [385]:
gdf_observ = gdf_observ.to_crs(31983)

In [386]:
gdf_observ

,SCM,geometry
0,3314-231,POINT (333344.322 7395282.674)
1,3331-132,POINT (324531.390 7385905.216)
2,4431-222,POINT (352135.100 7386243.508)


In [387]:
pt_obs = gdf_observ[gdf_observ.SCM == '3314-231'].geometry#.coords[:]

In [388]:
pt_obs = pt_obs[0].coords[:][0]

In [389]:
r = pdal.Pipeline(pipeline)
r.validate()
r.execute()
arrays = r.arrays

In [390]:
df = pd.DataFrame(arrays[0])

In [391]:
df_mds = \
df[(df.Classification == 2) | (df.Classification == 6)]\
                          [['X', 'Y', 'Z']]#\
# .astype({'X': 'int32', \
#         'Y': 'int32', \
#         'Z': 'int16'})

In [392]:
df_mdt = \
df[df.Classification == 2][['X', 'Y', 'Z']]

In [393]:
df

,X,Y,Z,Intensity,ReturnNumber,NumberOfReturns,ScanDirectionFlag,EdgeOfFlightLine,Classification,ScanAngleRank,UserData,PointSourceId,GpsTime
0,333499.29,7395564.35,735.34,18,1,1,0,0,6,-14.0,17,6,356569.366065
1,333499.23,7395565.79,735.41,18,1,1,0,0,6,-14.0,17,6,356569.366090
2,333499.20,7395566.79,735.25,15,1,1,0,0,6,-14.0,17,6,356569.366105
3,333499.15,7395568.21,735.37,14,1,1,0,0,6,-14.0,17,6,356569.366129
4,333499.10,7395569.68,735.30,17,1,1,0,0,6,-15.0,17,6,356569.366153
...,...,...,...,...,...,...,...,...,...,...,...,...,...
471957,333523.36,7395570.78,735.52,2,1,2,1,0,19,13.0,40,6,363044.324524
471958,333521.03,7395570.91,731.07,5,1,1,1,0,6,13.0,40,6,363044.342732
471959,333520.96,7395571.00,732.50,8,1,1,1,0,6,13.0,40,6,363044.342740
471960,333521.09,7395571.84,731.83,18,1,1,1,0,6,13.0,40,6,363044.360909


In [394]:
distance = np.sqrt((df_mdt.X - pt_obs[0])**2 + (df_mdt.Y - pt_obs[1])**2)

In [395]:
distance.sort_values()[:3].index

Int64Index([237624, 238187, 237180], dtype='int64')

In [396]:
df_mdt

,X,Y,Z
18,333499.25,7395550.72,724.76
20,333499.14,7395553.60,724.86
26,333498.88,7395560.92,724.87
38,333498.72,7395562.04,725.02
77,333498.36,7395551.34,724.89
...,...,...,...
471680,333563.96,7395552.14,724.30
471723,333558.16,7395554.04,724.33
471739,333556.44,7395554.74,724.53
471792,333530.38,7395555.62,724.47


## Inferindo o Z do ponto escolhido

In [397]:
pt_obs_z = df.iloc[distance.sort_values()[:3].index].Z.mean()

In [398]:
pt_obs_z

725.0866666666667

In [399]:
pt_obs

(333344.32172328787, 7395282.674381973)

## Dividindo o domo celeste

In [400]:
fusos = 60
meridianos = 30

In [401]:
thetas, delta_theta = np.linspace(-np.pi, np.pi, fusos, retstep=True, endpoint=False)
thetas

array([-3.14159265, -3.0368729 , -2.93215314, -2.82743339, -2.72271363,
       -2.61799388, -2.51327412, -2.40855437, -2.30383461, -2.19911486,
       -2.0943951 , -1.98967535, -1.88495559, -1.78023584, -1.67551608,
       -1.57079633, -1.46607657, -1.36135682, -1.25663706, -1.15191731,
       -1.04719755, -0.9424778 , -0.83775804, -0.73303829, -0.62831853,
       -0.52359878, -0.41887902, -0.31415927, -0.20943951, -0.10471976,
        0.        ,  0.10471976,  0.20943951,  0.31415927,  0.41887902,
        0.52359878,  0.62831853,  0.73303829,  0.83775804,  0.9424778 ,
        1.04719755,  1.15191731,  1.25663706,  1.36135682,  1.46607657,
        1.57079633,  1.67551608,  1.78023584,  1.88495559,  1.98967535,
        2.0943951 ,  2.19911486,  2.30383461,  2.40855437,  2.51327412,
        2.61799388,  2.72271363,  2.82743339,  2.93215314,  3.0368729 ])

In [402]:
phis = np.linspace(1., 0., meridianos, retstep=True, endpoint=False)
phis

(array([1.        , 0.96666667, 0.93333333, 0.9       , 0.86666667,
        0.83333333, 0.8       , 0.76666667, 0.73333333, 0.7       ,
        0.66666667, 0.63333333, 0.6       , 0.56666667, 0.53333333,
        0.5       , 0.46666667, 0.43333333, 0.4       , 0.36666667,
        0.33333333, 0.3       , 0.26666667, 0.23333333, 0.2       ,
        0.16666667, 0.13333333, 0.1       , 0.06666667, 0.03333333]),
 -0.03333333333333333)

In [403]:
np.arccos(phis[0])

array([0.        , 0.25892154, 0.36720802, 0.45102681, 0.52231482,
       0.58568554, 0.64350111, 0.69716313, 0.74758435, 0.79539883,
       0.84106867, 0.88494336, 0.92729522, 0.96834169, 1.00826008,
       1.04719755, 1.0852782 , 1.12260819, 1.15927948, 1.19537272,
       1.23095942, 1.26610367, 1.30086353, 1.33529209, 1.36943841,
       1.40334825, 1.43706474, 1.47062891, 1.50408018, 1.53745682])

## Discretizando $\theta$ e $\phi$

In [404]:
## Dado um angulo theta t em radianos qual seu indice em 
## uma divisão de fusos a cada delta_theta

t = np.pi - 0.000000001
t = np.arctan2(0,-1)
np.floor((t + np.pi)/delta_theta).astype('int')

60

In [405]:
## Dado um angulo phi f em radianos qual seu indice em
## uma divisão de meridianos

f = np.pi/2
indice = np.floor(meridianos - (np.cos(f) / (1 / meridianos))).astype('int')
indice

29

## Calculando $\theta$ e $\phi$ para todos os pontos

In [406]:
df_mds = df_mds.loc[df_mds.Z >= pt_obs_z]

In [407]:
df_mds['theta_idx'] = \
np.floor(np.arctan2(\
          df_mds.X - pt_obs[0],
          df_mds.Y - pt_obs[1])/delta_theta).astype('int')

In [442]:
df_mds['phi_idx'] = \
np.floor(meridianos - (np.cos(\
np.arctan2(\
           np.sqrt((df_mds.X - pt_obs[0])**2 + \
                   (df_mds.Y - pt_obs[1])**2), \
           df_mds.Z - pt_obs_z)) / \
                       (1 / meridianos))).astype('int')


In [444]:
df_mds.groupby(['theta_idx', 'phi_idx']).ngroups

1442

In [445]:
fusos * meridianos

1800

In [447]:
## FAtor de Visão de céu
1. - (df_mds.groupby(['theta_idx', 'phi_idx']).ngroups / (fusos * meridianos))

0.1988888888888889